In [4]:
#Adapted from Marc Joffe, 2016 

In [6]:
import os
import sys
import json
from backend import *#

In [92]:
tables_path = "./static/ug/muni_bonds/ER588705-ER457598-ER860368.pdf.txt.tables.json"
with codecs.open(tables_path, "r", "utf-8") as file:
    tables = json.load(file)

funds_table = tables['572']
schedule_table = tables['606']
#print json.dumps(schedule_table)

#subtypes = Counter(c[0]['subtype'] for c in schedule_table['data'] if c[0]['subtype'] != "none") 
#print subtypes
print schedule_table['sub_types']

[u'date', u'dollar', u'dollar', u'dollar', u'dollar', u'dollar']


In [79]:
txt_path = "./static/ug/muni_bonds/ER588705-ER457598-ER860368.pdf.txt"

import dateutil.parser as date_parser

def get_first_date(lines, query_string, threshold = 0.4):
    today = date.today()
    for i, l in enumerate(lines):
        if fuzzy_str_match(query_string, l) > threshold: 
            dt = date_parser.parse(l, fuzzy=True, default=today)
            if dt != today:
                return dt, i, l
            
            
with codecs.open(txt_path, 'r', 'utf-8', errors='replace') as file:
    print get_first_date(file, 'deliver')


(datetime.datetime(2012, 4, 26, 14, 33, 4, 45256), 50, u'delivery through DTC in New York, New York, on or about April 26, 2012.\n')


In [42]:
def find_row(table, query_string):
    #Find first 'other' typed column
    try:
        index = table['types'].index('other')
    except ValueError:
        print "no column with mainly string data found"
        return None
    
    strings = (s[index]['value'] for s in table['data'])
    scores_indices = ((val, idx) for (idx, val) in enumerate(fuzzy_str_match(query_string, s) for s in strings ) )
    
    return table['data'][max(scores_indices)[1]]

def closest_row_numeric_value(table, query_string):
    row = get_closest_row(table, query_string)
    if row:
        for r in row:
            if r['type'] in ('integer'): 
                return int(r['value'])
            elif r['type'] in ('large_num', 'small_float'):
                return float(r['value'].replace(",", ""))

In [46]:
def calc_net_proceeds(table):
    face_value = closest_row_numeric_value(table, 'Principal Amount')
    premium_or_discount = closest_row_numeric_value(table, 'Issue Premium')
    underwriter_discount = closest_row_numeric_value(table, 'Underwriter Discount')
    cost_of_issuance = closest_row_numeric_value(table, 'Costs of Issuance')    
    
    net_proceeds_calc = face_value + premium_or_discount - underwriter_discount - cost_of_issuance   
    return net_proceeds_calc

In [44]:
############################### MAIN CODE #############################

"""# Data from ESTIMATED SOURCES AND USES OF FUNDS
face_value = 2099309.10
premium_or_discount = 234391.95
underwriter_discount = 31489.64
cost_of_issuance = 145007.30

# Calculate Net Proceeds of the Bond Issue which is the first cashflow
net_proceeds_test = face_value + premium_or_discount - underwriter_discount - cost_of_issuance 
print net_proceeds_test

# Yields 2157204.11 for ER588705-ER457598-ER860368.pdf

"""

2157204.11


In [47]:

net_proceeds = calc_net_proceeds(funds_table)
print net_proceeds

2157204.11


In [50]:
import time
from datetime import date
from datetime import datetime

rate = None
dates = {}

# Delivery Date from text at the bottom of the cover page
delivery_date = [2012,4,26]


debug_each_guess = False  # Change to True for verbose output
guess = .05
guess_num = 1
guesses = []
discounted_cashflows = []

def newton(func, x0, fprime=None, args=(), tol=1.48e-8, maxiter=50):
    """Given a function of a single variable and a starting point,
    find a nearby zero using Newton-Raphson.

    fprime is the derivative of the function.  If not given, the
    Secant method is used.

    # Source: http://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.newton.html
    # File:   scipy.optimize.minpack.py
    # License: BSD: http://www.scipy.org/License_Compatibility
    """

    if fprime is not None:
        p0 = x0
        for iter in range(maxiter):
            myargs = (p0,)+args
            fval = func(*myargs)
            fpval = fprime(*myargs)
            if fpval == 0:
                print "Warning: zero-derivative encountered."
                return p0
            p = p0 - func(*myargs)/fprime(*myargs)
            if abs(p-p0) < tol:
                return p
            p0 = p
    else: # Secant method
        p0 = x0
        p1 = x0*(1+1e-4)
        q0 = func(*((p0,)+args))
        q1 = func(*((p1,)+args))
        for iter in range(maxiter):
            if q1 == q0:
                if p1 != p0:
                    print "Tolerance of %s reached" % (p1-p0)
                return (p1+p0)/2.0
            else:
                p = p1 - q1*(p1-p0)/(q1-q0)
            if abs(p-p1) < tol:
                return p
            p0 = p1
            q0 = q1
            p1 = p
            q1 = func(*((p1,)+args))
    raise RuntimeError, "Failed to converge after %d iterations, value is %s" % (maxiter,p)


def eir_func(rate, pmts, dates):
    """Loop through the dates and calculate a discounted cashflow total

    This is a simple process, but the debug messages clutter it up to
    make it seem more complex than it is.  With the debug messages removed,
    it is very similar to eir_derivative_func, but with the EIR formula,
    rather than f'rate.

    Credit: http://mail.scipy.org/pipermail/numpy-discussion/2009-May/042736.html
    """

    print_debug_messages = False
    
    # Globals used for debug printing
    global guess_num
    global debug_each_guess
    global guesses
    
    if rate not in guesses:
        print_debug_messages = debug_each_guess
        guesses.append(rate)
        if print_debug_messages:
            print "-----------------------------------------------------------------------------------------------"
            print "Guess #%s:  %s" % (guess_num, rate)
            print ""
            print "   # DATE          # DAYS  CASHFLOW      DISCOUNTED    Formula: cf * (rate + 1)^(-days/365)"
            print "   --------------------------------------------------------------------------------------------"
        guess_num +=1

    dcf=[]
    for i, cf in enumerate(pmts):
        d = dates[i] - dates[0]
        discounted_period = cf * (rate + 1)**(-d.days / 365.)
        dcf.append( discounted_period )

        if print_debug_messages:
            cf = "%.2f" % cf
            cf = cf.rjust(9, " ")
            discounted_period = '%.8f' % discounted_period
            formula = '%s * ((%0.10f + 1)^(-%d /365)) ' % (cf, rate, d.days)
            discounted_period = discounted_period.rjust(15, " ")
            print "  %2i %s  %3.0d days %s %s =%s"  % \
            (i, dates[i], d.days, cf, discounted_period, formula )

    discounted_cashflow = sum(dcf)

    if print_debug_messages:
        discounted_cashflow = "%.8f" % discounted_cashflow
        total = "total:".rjust(35, " ")
        print "%s %s" % (total, discounted_cashflow.rjust(15, " "))
        print ""

    return discounted_cashflow

def eir_derivative_func(rate, pmts, dates):
    """Find the derivative or the EIR function, used for calculating
    Newton's method:

    http://en.wikipedia.org/wiki/Newton's_method

    EIR = cf*(1+rate)^d
    f'rate = cf*d*(rate+1)^(d-1)

    Credit: http://mail.scipy.org/pipermail/numpy-discussion/2009-May/042736.html
    """

    dcf=[]
    for i, cf in enumerate(pmts):
        d = dates[i] - dates[0]
        n = (-d.days / 365.)
        dcf.append( cf * n * (rate + 1)**(n - 1) )
    return sum(dcf)




In [49]:
cnt = 0 
rate = None
dates = {}

# Delivery Date from text at the bottom of the cover page
delivery_date = [2012,4,26]

payments = [
net_proceeds,
-57033.85,
-37331.25,
-37331.25,
-37331.25,
-37331.25,
-37331.25,
-37331.25,
-37331.25,
-37331.25,
-37331.25,
-37331.25,
-37331.25,
-37331.25,
-37331.25,
-37331.25,
-52331.25,
-37331.25,
-52331.25,
-37331.25,
-57331.25,
-37331.25,
-62331.25,
-37331.25,
-67331.25,
-37331.25,
-72331.25,
-37331.25,
-72331.25,
-37331.25,
-82331.25,
-37331.25,
-82331.25,
-37331.25,
-87331.25,
-37331.25,
-92331.25,
-37331.25,
-102331.25,
-37331.25,
-107331.25,
-37331.25,
-112331.25,
-37331.25,
-122331.25,
-37331.25,
-122331.25,
-37331.25,
-132331.25,
-37331.25,
-457331.25,
-37331.25,
-472331.25,
-37331.25,
-492331.25,
-32278.13,
-512278.13,
-22378.13,
-542378.13,
-11653.13,
-576653.13,
]

payment_dates=[
delivery_date,
[2013,2,1],
[2013,8,1],
[2014,2,1],
[2014,8,1],
[2015,2,1],
[2015,8,1],
[2016,2,1],
[2016,8,1],
[2017,2,1],
[2017,8,1],
[2018,2,1],
[2018,8,1],
[2019,2,1],
[2019,8,1],
[2020,2,1],
[2020,8,1],
[2021,2,1],
[2021,8,1],
[2022,2,1],
[2022,8,1],
[2023,2,1],
[2023,8,1],
[2024,2,1],
[2024,8,1],
[2025,2,1],
[2025,8,1],
[2026,2,1],
[2026,8,1],
[2027,2,1],
[2027,8,1],
[2028,2,1],
[2028,8,1],
[2029,2,1],
[2029,8,1],
[2030,2,1],
[2030,8,1],
[2031,2,1],
[2031,8,1],
[2032,2,1],
[2032,8,1],
[2033,2,1],
[2033,8,1],
[2034,2,1],
[2034,8,1],
[2035,2,1],
[2035,8,1],
[2036,2,1],
[2036,8,1],
[2037,2,1],
[2037,8,1],
[2038,2,1],
[2038,8,1],
[2039,2,1],
[2039,8,1],
[2040,2,1],
[2040,8,1],
[2041,2,1],
[2041,8,1],
[2042,2,1],
[2042,8,1],
]

# Convert our list of dates into date types
for i,dt in enumerate(payment_dates):
    dates[i]=date(*dt)

# Begin Main Calculation
timer_start = time.clock()
if len(dates) > 1:
    f = lambda x: eir_func(x, payments, dates)
    derivative = lambda x: eir_derivative_func(x, payments, dates)
    try:
        rate = newton(f, guess, fprime=derivative, args=(),
            tol=0.00000000001, maxiter=100)
    except RuntimeError:
        pass # failed to converge after maxiterations

timer_end = time.clock()
# End Main Calculation

elapsed_time = timer_end - timer_start
final_rate = rate * 100

if not debug_each_guess:
    print ""
    print "Cashflow Dates: "
    print "-------------------------"
    for i, dte in enumerate(payment_dates):
        print "%s %s " % (date(*dte), str(payments[i]).rjust(5," ") )

print """
Guesses Summary
------------------"""

for i, guess in enumerate(guesses):
  print i +1, "%0.10f" % guess

print """
Final Rate: %.5f %%
""" % final_rate

print """Calculation time: %s seconds
""" % elapsed_time


Cashflow Dates: 
-------------------------
2012-04-26 2157204.11 
2013-02-01 -57033.85 
2013-08-01 -37331.25 
2014-02-01 -37331.25 
2014-08-01 -37331.25 
2015-02-01 -37331.25 
2015-08-01 -37331.25 
2016-02-01 -37331.25 
2016-08-01 -37331.25 
2017-02-01 -37331.25 
2017-08-01 -37331.25 
2018-02-01 -37331.25 
2018-08-01 -37331.25 
2019-02-01 -37331.25 
2019-08-01 -37331.25 
2020-02-01 -37331.25 
2020-08-01 -52331.25 
2021-02-01 -37331.25 
2021-08-01 -52331.25 
2022-02-01 -37331.25 
2022-08-01 -57331.25 
2023-02-01 -37331.25 
2023-08-01 -62331.25 
2024-02-01 -37331.25 
2024-08-01 -67331.25 
2025-02-01 -37331.25 
2025-08-01 -72331.25 
2026-02-01 -37331.25 
2026-08-01 -72331.25 
2027-02-01 -37331.25 
2027-08-01 -82331.25 
2028-02-01 -37331.25 
2028-08-01 -82331.25 
2029-02-01 -37331.25 
2029-08-01 -87331.25 
2030-02-01 -37331.25 
2030-08-01 -92331.25 
2031-02-01 -37331.25 
2031-08-01 -102331.25 
2032-02-01 -37331.25 
2032-08-01 -107331.25 
2033-02-01 -37331.25 
2033-08-01 -112331.25 
2034-0